In [1]:
import pandas as pd
import numpy as np

from context import community_module
from community_module.community_detection.similarityCommunityDetection import SimilarityCommunityDetection
from community_module.community_detection.explainedCommunitiesDetection import ExplainedCommunitiesDetection

from sklearn.metrics.pairwise import cosine_similarity

import uuid

In [2]:
data_df = pd.read_csv('../../data/MNCN/user_profiles.csv')

# Datos necesarios para los filtros
q103_index = list(range(1,7))
q107a_index = list(range(7, 13))
q107b_index = list(range(13, 16))
q110a_index = list(range(16, 18))
q110b_index = list(range(18, len(data_df.columns)))

# Questions
questions = {
    'q103': q103_index,
    'q107a': q107a_index,
    'q107b': q107b_index,
    'q110a': q110a_index,
    'q110b': q110b_index
}

# Diccionario de respuestas
answers = {
    'q103_0': 'Reduce shower time',
    'q103_1': 'Buying less clothes',
    'q103_2': 'Do not use products with a lot of packaging',
    'q103_3': 'Walking to more places',
    'q103_4': 'Reduce my waste generation',
    'q103_5': 'Recycle correctly',
    'q107a_0': 'Transport: By car',
    'q107a_1': 'Transport: Walking',
    'q107a_2': 'Transport: By bike',
    'q107a_3': 'Transport: By bus',
    'q107a_4': 'Transport: By underground',
    'q107a_5': 'Transport: By En scooter',
    'q107b_0': 'I would be willing to change means of transport',
    'q107b_1': 'I would not be willing to change means of transport',
    'q107b_2': 'Perhaps I would be willing to change means of transport',
    'q110a_0': 'I had an exotic pet',
    'q110a_1': 'I did not have an exotic pet',
    'q110b_0': 'Adopted: florida tortoise',
    'q110b_1': 'Adopted: common dog',
    'q110b_2': 'Adopted: common cat',
    'q110b_3': 'Adopted: Argentine parrot',
    'q110b_4': 'Adopted: cockatoo',
    'q110b_5': 'Adopted: capuchin monkey',
}

# Ejemplos resultados para la API

En este notebook mostraremos los resultados que debe devolver la API del Community Model. Los ejemplos se obtienen de resultados reales aplicados al ejemplo del Museo Nacional de Ciencias Naturales.

## Communities

- `GET /communities`:

In [6]:
indexes = list(range(1, 16))
data = data_df.iloc[:,indexes]

community_detection = ExplainedCommunitiesDetection(data, SimilarityCommunityDetection, 'cosine')
n_communities, users_communities = community_detection.search_all_communities(answer_binary=True, percentage=1.0)

result = list()

for c in range(n_communities):
    community_data = community_detection.get_community(c)
    
    if len(community_data['members']) > 1:
        com = dict()
        com['id'] = str(uuid.uuid1())
        com['name'] = 'MNCN-{}'.format(community_data['name'])
        com['size'] = len(community_data['members'])
        com['members'] = community_data['members']
        com['community-type'] = 'implicit'
        com['explanations'] = []
        
        for k in community_data['properties'].keys():
            com['explanations'].append(answers[k])
            
        result.append(com)
        
result

[{'id': '9e2e3e56-7aa9-11ec-9c5d-acbc32b03991',
  'name': 'MNCN-0',
  'size': 2,
  'members': [38, 42],
  'community-type': 'implicit',
  'explanations': ['Reduce shower time',
   'Walking to more places',
   'Reduce my waste generation',
   'Recycle correctly',
   'Transport: Walking',
   'Transport: By bike',
   'Transport: By bus',
   'Transport: By underground',
   'I would be willing to change means of transport',
   'I would not be willing to change means of transport',
   'Perhaps I would be willing to change means of transport']},
 {'id': '9e2f591c-7aa9-11ec-9c5d-acbc32b03991',
  'name': 'MNCN-1',
  'size': 6,
  'members': [25, 39, 43, 48, 50, 53],
  'community-type': 'implicit',
  'explanations': ['Buying less clothes',
   'Walking to more places',
   'Reduce my waste generation',
   'Recycle correctly',
   'Transport: By car',
   'Transport: By bike',
   'Transport: By underground',
   'Transport: By En scooter',
   'I would be willing to change means of transport']},
 {'id':

- `GET /communities/{communityId}`:

In [7]:
result[2]

{'id': '9e307b3a-7aa9-11ec-9c5d-acbc32b03991',
 'name': 'MNCN-2',
 'size': 6,
 'members': [16, 24, 27, 30, 44, 54],
 'community-type': 'implicit',
 'explanations': ['Buying less clothes',
  'Do not use products with a lot of packaging',
  'Transport: By car',
  'Transport: By bike',
  'Transport: By underground',
  'Transport: By En scooter',
  'I would be willing to change means of transport',
  'I would not be willing to change means of transport',
  'Perhaps I would be willing to change means of transport']}

- `GET /communities/{communityId}/users`

In [8]:
result[2]['members']

[16, 24, 27, 30, 44, 54]

## Users

- `GET /users/{userId}/communities` 

In [12]:
coms = []
for r in result[:2]:
    coms.append(r['id'])
    
coms

['9e2e3e56-7aa9-11ec-9c5d-acbc32b03991',
 '9e2f591c-7aa9-11ec-9c5d-acbc32b03991']

## Similarity

- `GET /communities/{communityId}/similarity`
- `GET /communities/{communityId}/similarity/{otherCommunityId}`
- `GET /communities/{communityId}/dissimilarity`
- `GET /communities/{communityId}/dissimilarity/{otherCommunityId}`